In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from groupy.gconv.pytorch_gconv.splitgconv2d import P4ConvZ2, P4ConvP4
from groupy.gconv.pytorch_gconv.pooling import plane_group_spatial_max_pooling

In [5]:



# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args("")
#args ={}

args.lr=0.01
args.momentum=0.5
args.epochs=10
args.seed=1
args.batch_size=64
args.test_batch_size=64

args.cuda = not args.no_cuda and torch.cuda.is_available()

In [8]:


torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = P4ConvZ2(1, 10, kernel_size=3)
        self.conv2 = P4ConvP4(10, 10, kernel_size=3)
        self.conv3 = P4ConvP4(10, 20, kernel_size=3)
        self.conv4 = P4ConvP4(20, 20, kernel_size=3)
        #self.fc1 = nn.Linear(4*4*20*4, 50)
        self.fc2 = nn.Linear(4*4*20*4, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = plane_group_spatial_max_pooling(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = plane_group_spatial_max_pooling(x, 2, 2)
        x = x.view(x.size()[0], -1)
        #x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x,dim=1)

model = Net()
if args.cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()


/home/facuq/dev/exp/rotation/.env/lib/python3.6/site-packages/ipykernel_launcher.py:68: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309609
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.007188
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.167897
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.000239
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.421772
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.501148
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.458267
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.266348
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.126960
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.305199
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.269154
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.151796
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.390512
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.178355
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.123175
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.163192
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.168039
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.125321
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.108268
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.087680
Train Epoch: 1 [12800/60000 (

/home/facuq/dev/exp/rotation/.env/lib/python3.6/site-packages/ipykernel_launcher.py:78: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/facuq/dev/exp/rotation/.env/lib/python3.6/site-packages/ipykernel_launcher.py:80: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0538, Accuracy: 9821/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.060180
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.046189
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.073419
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.008092
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.204784
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.117283
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.006516
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.012829
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.067952
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.008006
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.048662
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.039161
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.016568
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.040572
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.174925
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.047783
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.067005
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.015030
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.024357
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.006596
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.062181
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.059830
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.027872
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.089456
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.048321
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.008757
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.006834
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.009436
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.046906
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.004313
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.001899
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.047623
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.007239
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.013428
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.004138
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.002720
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.050035
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.057268
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.031346


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.021809
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.000746
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.005528
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.004471
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.033906
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.026116
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.049618
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.033892
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.033115
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.002594
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.006056
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.016845
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.008106
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.011884
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.003757
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.010867
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.060326
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.105283
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.024275
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.004591


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.000704
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.014756
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.001880
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.009105
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.009338
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.015462
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.009191
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.028270
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.004553
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.012336
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.081556
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.044075
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.014130
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.002941
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.004595
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.022498
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.139989
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.000476
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.000953
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.012311


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.000544
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.006492
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.003620
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.003660
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.045637

Test set: Average loss: 0.0246, Accuracy: 9921/10000 (99%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.007641
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.001230
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.019658
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.041911
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.007018
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.020774
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.008583
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.004727
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.011653
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.002365
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.002766
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.054432
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.123068
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.008629
Train Epoch: 9 [

Train Epoch: 10 [40320/60000 (67%)]	Loss: 0.002524
Train Epoch: 10 [40960/60000 (68%)]	Loss: 0.002292
Train Epoch: 10 [41600/60000 (69%)]	Loss: 0.004428
Train Epoch: 10 [42240/60000 (70%)]	Loss: 0.005044
Train Epoch: 10 [42880/60000 (71%)]	Loss: 0.001096
Train Epoch: 10 [43520/60000 (72%)]	Loss: 0.029122
Train Epoch: 10 [44160/60000 (74%)]	Loss: 0.001893
Train Epoch: 10 [44800/60000 (75%)]	Loss: 0.004598
Train Epoch: 10 [45440/60000 (76%)]	Loss: 0.005165
Train Epoch: 10 [46080/60000 (77%)]	Loss: 0.009857
Train Epoch: 10 [46720/60000 (78%)]	Loss: 0.001133
Train Epoch: 10 [47360/60000 (79%)]	Loss: 0.006985
Train Epoch: 10 [48000/60000 (80%)]	Loss: 0.000699
Train Epoch: 10 [48640/60000 (81%)]	Loss: 0.002501
Train Epoch: 10 [49280/60000 (82%)]	Loss: 0.001005
Train Epoch: 10 [49920/60000 (83%)]	Loss: 0.000264
Train Epoch: 10 [50560/60000 (84%)]	Loss: 0.001701
Train Epoch: 10 [51200/60000 (85%)]	Loss: 0.001136
Train Epoch: 10 [51840/60000 (86%)]	Loss: 0.001223
Train Epoch: 10 [52480/60000 (8